# 特征选择
单变量选择法的主要思路是根据某些统计检验的方法分别对每个变量进行检验，得到一组分数、p-value数据，然后我们排序选择分数最高(或p-value最小等)的那些特征。在sklearn中，分别针对不同的统计检验方法和不同的排序选择标准提供了不同的工具，比如用于回归问题的f_regression、mutual_info_regression分数，用于分类问题的f_classif、chi2、mutual_info_classf分数；以及用于特征排序和选择的SelectKBest、SelectPercentile、SelectFpr等。把这两类工具相结合，就可以完成特征选择的任务了。

## SelectKBest
顾名思义，SelectKBest方法就是从所有特征中挑选出最好的K个特征组成新的特征集。如何来定义最好呢？这要取决于我们传入的评分函数，该方法默认会使用f_classif，这种方法适用于为分类模型评估特征，与之对应的是f_regression，它适用于回归模型的特征评估。

In [24]:
from sklearn.datasets import load_boston
from sklearn.feature_selection import SelectKBest, f_regression, SelectFpr
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import SelectFdr
from sklearn.feature_selection import SelectFwe
from sklearn.feature_selection import GenericUnivariateSelect

这里我们导入了波士顿房价的数据集，该数据集是一个回归问题，共有506个样本、13个特征，我们的任务是根据这13个特征来预测房价。我们先使用所有的特征数据来训练一个模型，作为一个基准。

In [3]:
boston = load_boston()
X = boston.data
y = boston.target
print(X.shape)
print(y.shape)


(506, 13)
(506,)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1001)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print('R2: ', r2)
print('MSE: ', mse)

R2:  0.6783942923302064
MSE:  29.824006898863125


那么接下来我们就从这13个特征中选择最好的10个特征来训练一个新的模型

先生成新的特征子集


In [5]:
# k=10 选择10个
selector = SelectKBest(f_regression, k = 10)
X_new = selector.fit_transform(X, y)
print(X_new.shape)

(506, 10)


可以看到，样本数不变，仍为506个，特征则被删减到了10个。接下来我们用这10个特征的数据来训练一个新的模型。注意，在使用train_test_split方法分割测试集和训练集时，我们要保证random_state参数的取值是固定的，这样才能确保不同的模型训练和测试过程中，训练集和测试集的样本是一致的。

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, random_state = 1001)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print('R2: ', r2)
print('MSE: ', mse)

R2:  0.6826443789276992
MSE:  29.42987641864622


我们发现在我们删掉了3个特征以后,R2略有提升，均方误差MSE略有下降，也就是说我们模型的效果反而更好了。

## SelectPercentile

SelectPercentile的选取方式与SelectKBest略有不同，后者如我们之前所示，是选取在某种标准下最好的K个特征；而前者，则是在给定一个0到1之间的百分比x%的情况下，选择最好的x%的特征。比如10个特征，前20%的特征就是最好的两个特征，以此类推。

其使用的评分标准与SelectKBest没有什么不同，按需传入所需的评分方法即可。需要注意的是，percentile接受的是一个0到100的整数。

In [10]:
selector = SelectPercentile(f_regression, percentile=70)
X_new = selector.fit_transform(X, y)
print(X_new.shape)

(506, 9)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, random_state=1001)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print('R2: ', r2)
print('MSE: ', mse)

R2:  0.6831525503257942
MSE:  29.382751299529392


可以看到，我们选择了前70%的特征(9个)，模型的R2进一步提升。

## SelectFpr

前两种方法都很好理解，而SelectFpr方法则要求对假设检验有一定的认识。这种方法基于FPR测试，即False Positive Rate，其含义为假阳性率，是指被我们预测为正但实际为负的样本的比例，也就是假设检验中的一类错误发生的比例。

不想深究的同学可以直接简单粗暴地记住，这里就是按照p-value来筛选特征的，p-value越小越好，所有p-value低于我们设定的阈值α的特征都会被选择。由于这个例子中每个特征的p-value都很小，为了演示效果，我们把α设定到十万分之一：

In [27]:
selector = SelectFpr(f_regression, alpha=0.00001)
X_new = selector.fit_transform(X, y)
print(X_new.shape)

(506, 12)


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, random_state=1001)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print('R2: ', r2)
print('MSE: ', mse)

R2:  0.6935405006596644
MSE:  28.419427903725463


In [15]:
selector.get_support()

array([ True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

可以看到，f_regression检验中，p值在十万分之一以下的特征（第四个）被剔除了

## SelectFdr
SelectFdr则是根据错误发现率（FDR，False Discovery Rate）来为特征排序。错误发现率与假阳性率非常相似，但是却有着本质的不同。假阳性率是在所有样本中一类错误发生的概率，而错误发现率则仅仅关注在我们拒绝原假设(预测为正)的样本中，有多大比例是犯了一类错误的（即在"发现"的样本里，有多少是错误的"发现"）。

sklearn中的SelectFdr方法使用Benjamini-Hochberg过程对p值进行修正，并筛选出修正后的p值在α水平以下的特征。

在这个例子中，SelectFdr和SelectFpr的结果是一样的。

In [18]:
selector = SelectFdr(f_regression, alpha=0.00001)
X_new = selector.fit_transform(X, y)
print(X_new.shape)

(506, 12)


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, random_state=1001)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print('R2: ', r2)
print('MSE: ', mse)

R2:  0.6935405006596644
MSE:  28.419427903725463


## SelectFwe
在多重假设检验中，总体错误率(族系误差率，FWER，family-wise error rate)是另一个常用的错误控制指标，它与FDR的区别在于，FWER是指至少出现一次一类错误的概率，而FDR则是关注预测为正的样本中一类错误发生的比例。

举个例子来帮助大家理解FPR、FDR和FWER之间的区别，共有20个样本，我们做了100次多重检验，其中有30次出现了一类错误，这30次中，平均每次检验中我们会拒绝\H_010次，其中2次是错误拒绝。这时：

```
FPR = (2*30) / (20*100) = 0.03
FDR = (2*30) / (10*30) = 0.2
FWER = 3/10 = 0.3
```

当然，在sklearn中是直接使用特定的过程对p值进行修正，并不是真正地去进行多次模拟并计算各项指标。

我们来看下sklearn中的SelectFwe的使用：

In [21]:
selector = SelectFwe(f_regression, alpha=0.0000001)
X_new = selector.fit_transform(X, y)
print(X_new.shape)

(506, 11)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, random_state=1001)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print('R2: ', r2)
print('MSE: ', mse)

R2:  0.6906191228974212
MSE:  28.69034097665135


在不同的α下，SelectFpr、SelectFdr和SelectFwe所选取的特征会出现不同，大家可以自行测试一下。

## GenericUnivariateSelect

这种方法将上述五种方法集成到了一起，然后我们将评分方法、选择方法以参数的形式传递进来即可。

In [25]:
selector = GenericUnivariateSelect(f_regression, mode='fpr', param=0.0000001)
X_new = selector.fit_transform(X, y)
print(X_new.shape)

(506, 12)


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, random_state=1001)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print('R2: ', r2)
print('MSE: ', mse)

R2:  0.6935405006596644
MSE:  28.419427903725463
